In [1]:
import sys
!{sys.executable} -m pip install pyspark

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit

In [3]:
# Создаем Spark сессию
spark = SparkSession.builder.master("local").appName("ProductCategory").getOrCreate()

24/07/24 10:28:01 WARN Utils: Your hostname, mista-ROG-Strix-G614JV-G614JV resolves to a loopback address: 127.0.1.1; using 192.168.16.101 instead (on interface wlo1)
24/07/24 10:28:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/24 10:28:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Пример данных
products_data = [("Product1",), ("Product2",), ("Product3",)]
categories_data = [("Category1",), ("Category2",)]
product_category_data = [("Product1", "Category1"), ("Product2", "Category2")]


In [5]:
# Создаем датафреймы
products_df = spark.createDataFrame(products_data, ["product_name"])
categories_df = spark.createDataFrame(categories_data, ["category_name"])
product_category_df = spark.createDataFrame(product_category_data, ["product_name", "category_name"])

In [8]:
# Метод для получения пар и продуктов без категорий
def get_product_category_info(products_df, categories_df, product_category_df):
    # Внешнее соединение, чтобы получить все продукты с категориями и без
    product_category_joined = products_df.join(product_category_df, "product_name", "left_outer")
    
    # Продукты без категорий
    products_without_category = product_category_joined.filter(col("category_name").isNull()).select("product_name")
    
    # Продукты с категориями
    product_category_pairs = product_category_joined.filter(col("category_name").isNotNull()).select("product_name", "category_name")
    
    return product_category_pairs, products_without_category

In [9]:
# Применяем метод
product_category_pairs, products_without_category = get_product_category_info(products_df, categories_df, product_category_df)

In [10]:
# Выводим результат
print("Product-Category Pairs:")
product_category_pairs.show()

Product-Category Pairs:
+------------+-------------+
|product_name|category_name|
+------------+-------------+
|    Product1|    Category1|
|    Product2|    Category2|
+------------+-------------+



In [11]:
print("Products without Category:")
products_without_category.show()

Products without Category:
+------------+
|product_name|
+------------+
|    Product3|
+------------+

